In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 7.4MB/s 
     |████████████████████████████████| 1.2MB 14.6MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 2.3MB 47.6MB/s 
     |████████████████████████████████| 19.7MB 328kB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 798kB 30.8MB/s 
     |████████████████████████████████| 983kB 26.6MB/s 
     |████████████████████████████████| 3.3MB 28.0MB/s 
     |████████████████████████████████| 901kB 45.1MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=adc3e783f432c9c588a76a0b198b2a35fecc3a0199cd6e1ba56a44c84ba43b94
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for ftfy: filename=f

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/NLP/2_Competição/Dataset'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file= 'train.conll',
                              test_file= 'test.conll')

2021-05-25 16:46:33,910 Reading data from /content/drive/MyDrive/NLP/2_Competição/Dataset
2021-05-25 16:46:33,912 Train: /content/drive/MyDrive/NLP/2_Competição/Dataset/train.conll
2021-05-25 16:46:33,916 Dev: None
2021-05-25 16:46:33,917 Test: /content/drive/MyDrive/NLP/2_Competição/Dataset/test.conll


In [ ]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from torch.optim.adam import Adam

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # GloVe embeddings
    WordEmbeddings('pt'),

    # # contextual string embeddings, forward
    PooledFlairEmbeddings('pt-forward', pooling='min'),

    # # contextual string embeddings, backward
    PooledFlairEmbeddings('pt-backward', pooling='min'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        )

# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/nerFinal',
              train_with_dev=False,
              learning_rate=0.1,
              min_learning_rate=0.0001,
              mini_batch_size=32,
              max_epochs=110)


2021-05-25 16:47:07,256 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp938krfxq


100%|██████████| 710528528/710528528 [00:40<00:00, 17513318.88B/s]

2021-05-25 16:47:48,356 copying /tmp/tmp938krfxq to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M.vectors.npy


2021-05-25 16:47:50,112 removing temp file /tmp/tmp938krfxq
2021-05-25 16:47:51,017 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmplzm2ynag


100%|██████████| 23541010/23541010 [00:02<00:00, 10428809.36B/s]

...

2021-05-25 16:48:02,182 removing temp file /tmp/tmpdiigkoo9
2021-05-25 16:48:06,350 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-backward.pt not found in cache, downloading to /tmp/tmpwksfs303


100%|██████████| 72819080/72819080 [00:04<00:00, 14713207.02B/s]

2021-05-25 16:48:11,823 copying /tmp/tmpwksfs303 to cache at /root/.flair/embeddings/lm-pt-backward.pt


2021-05-25 16:48:11,918 removing temp file /tmp/tmpwksfs303
2021-05-25 16:48:13,852 ----------------------------------------------------------------------------------------------------
2021-05-25 16:48:13,855 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('pt')
    (list_embedding_1): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.5, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=275, bias=True)
        )
      )
    )
    (list_embedding_2): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.5, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=275, bias=True)
        )
      )
    )
  )

{'dev_loss_history': [119.4624252319336,
  53.648963928222656,
  43.50716781616211,
  38.04669189453125,
  38.63785934448242,
  37.14254379272461,
  42.007301330566406,
  51.693172454833984,
  32.06605529785156,
  41.12238311767578,
  44.57950973510742,
  45.47738265991211,
  36.927947998046875,
  39.826873779296875,
  31.797462463378906,
  60.07123947143555,
  47.66645431518555,
  65.44149780273438,
  90.24217224121094,
  49.13070297241211,
  39.0938606262207,
  31.614105224609375,
  49.83608627319336,
  26.705989837646484,
  67.32825469970703,
  29.958568572998047,
  22.474288940429688,
  26.53125762939453,
  21.26583480834961,
  16.248334884643555,
  15.936168670654297,
  20.31825828552246,
  20.81747817993164,
  10.8062162399292,
  7.571802139282227,
  14.405611991882324,
  9.285595893859863,
  10.271820068359375,
  10.610743522644043,
  9.938639640808105,
  7.990126609802246,
  7.164840221405029,
  8.824713706970215,
  7.026019096374512,
  6.580907344818115,
  7.887789726257324,
 

In [ ]:
final = pd.read_csv("/content/resources/taggers/nerFinal/test.tsv", sep = ' ', skip_blank_lines=False,header=None)

In [ ]:
final2 = final

In [ ]:
final2 = final2.drop(columns=[1,2])

In [ ]:
final2['Tag'] = final[2]

In [ ]:
final2 = final2.drop(columns=[0])

In [ ]:
final2.index

RangeIndex(start=0, stop=14079, step=1)

In [ ]:
final2["Id"] = pd.Series(np.arange(start=1, stop=14080, step=1), final2.index)

In [ ]:
cols = final2.columns.tolist()
print(cols)
cols = cols[-1:] + cols[:-1]
print(cols)
final2 = final2[cols]

['Tag', 'Id']
['Id', 'Tag']


In [ ]:
final2 = final2.dropna()

In [ ]:
final2.head(30)

,Id,Tag
0,1,O
1,2,O
2,3,O
4,5,O
5,6,O
6,7,O
7,8,O
8,9,O
10,11,O
11,12,O


In [ ]:
final2.to_csv('/content/drive/MyDrive/NLP/2_Competição/resultados/stackedWePtLSTMAdam.csv',index=False)

In [ ]:
from google.colab import files
files.download('/content/resources/taggers/example-ner/final-model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>